In [1]:
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time

In [2]:
airport_staids = {
    'JFK': 74486
    ,'MCO': 72205
    ,'TPA': 72211
           }

In [4]:
period_start = "2024-08-01"
period_end = "2024-08-31"

config = dotenv_values()
api_key = config['RAPIDAPI_key']

In [5]:
weather_dict = {'extracted_at':[], 
                'airport_code':[], 
                'station_id':[], 
                'extracted_data':[]
               }

# API CALL daily (station) - for the syntax: see the rapidapi interface

url = "https://meteostat.p.rapidapi.com/stations/daily"

headers = {                   #headers dictionary is use to send specific information as part of the HTTPS request
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# for-loop for the querystrings
for airport in airport_staids:
   
    querystring = {
        "station":airport_staids[airport]
        ,"start":period_start
        ,"end":period_end
        ,"model":"true"
    }
    
    # making one call with the current querystring
    response = requests.get(url, headers=headers, params=querystring)  #This argument is where you pass additional parameters to the API in the form of a dictionary 
                
    # appending data to the dictionary:
    weather_dict['extracted_at'].append(datetime.now())                # timestamp, 
    weather_dict['airport_code'].append(airport)                       # airport code    
    weather_dict['station_id'].append(airport_staids[airport])         # weater Station ID
    weather_dict['extracted_data'].append(json.loads(response.text))   # JSON string

    time.sleep(0.5)

In [6]:
weather_daily_df = pd.DataFrame(weather_dict)
weather_daily_df

,extracted_at,airport_code,station_id,extracted_data
0,2025-03-20 18:41:27.600382,JFK,74486,"{'meta': {'generated': '2025-03-20 17:41:27'},..."
1,2025-03-20 18:41:28.329448,MCO,72205,"{'meta': {'generated': '2025-03-20 17:41:28'},..."
2,2025-03-20 18:41:29.053129,TPA,72211,"{'meta': {'generated': '2025-03-20 17:41:29'},..."


In [8]:
pd.json_normalize(weather_daily_df['extracted_data'][0], record_path='data')

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2024-08-01 00:00:00,28.4,23.9,33.9,0.0,0.0,220.0,14.8,None,1013.6,None
1,2024-08-02 00:00:00,28.2,23.9,32.8,4.6,0.0,180.0,15.1,None,1011.7,None
2,2024-08-03 00:00:00,27.0,23.3,31.1,32.5,0.0,214.0,17.3,None,1010.1,None
3,2024-08-04 00:00:00,24.4,22.2,27.2,4.6,0.0,224.0,16.2,None,1013.3,None
4,2024-08-05 00:00:00,26.6,22.2,31.7,0.0,0.0,236.0,14.4,None,1015.7,None
5,2024-08-06 00:00:00,26.4,22.2,30.6,21.1,0.0,193.0,14.8,None,1013.6,None
6,2024-08-07 00:00:00,21.2,18.9,22.8,1.5,0.0,62.0,16.9,None,1015.1,None
7,2024-08-08 00:00:00,20.2,18.9,23.3,7.1,0.0,74.0,19.4,None,1018.3,None
8,2024-08-09 00:00:00,24.6,22.8,28.3,6.6,0.0,156.0,31.7,None,1012.5,None
9,2024-08-10 00:00:00,26.4,23.3,30.6,0.0,0.0,280.0,19.4,None,1012.7,None


In [10]:
config = dotenv_values()
 
pg_user = config['POSTGRES_USER'] # align the key labels with your .env file
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [11]:
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

engine = create_engine(url, echo=False)

In [12]:
dtype_dict = {
    'extracted_at':types.DateTime,
    'airport_code': types.String,
    'station_id': types.Integer,
    'extracted_data':postgres_json
             }

In [13]:
weather_daily_df.to_sql(name = 'weather_daily_raw', 
                       con = engine, 
                       schema = pg_schema,
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

3